# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report


url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [28]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_respone', engine)
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [14]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)


In [58]:
y_train.mean()

related                   1.0
request                   0.0
offer                     0.0
aid_related               0.0
medical_help              0.0
medical_products          0.0
search_and_rescue         0.0
security                  0.0
military                  0.0
child_alone               0.0
water                     0.0
food                      0.0
shelter                   0.0
clothing                  0.0
money                     0.0
missing_people            0.0
refugees                  0.0
death                     0.0
other_aid                 0.0
infrastructure_related    0.0
transport                 0.0
buildings                 0.0
electricity               0.0
tools                     0.0
hospitals                 0.0
shops                     0.0
aid_centers               0.0
other_infrastructure      0.0
weather_related           0.0
floods                    0.0
storm                     0.0
fire                      0.0
earthquake                0.0
cold      

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [53]:
pipeline.fit(X_train, y_train)



Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ffab6ad9b00>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [54]:
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [55]:
np.mean(y_pred == y_test)

related                   1.0
request                   1.0
offer                     1.0
aid_related               1.0
medical_help              1.0
medical_products          1.0
search_and_rescue         1.0
security                  1.0
military                  1.0
child_alone               1.0
water                     1.0
food                      1.0
shelter                   1.0
clothing                  1.0
money                     1.0
missing_people            1.0
refugees                  1.0
death                     1.0
other_aid                 1.0
infrastructure_related    1.0
transport                 1.0
buildings                 1.0
electricity               1.0
tools                     1.0
hospitals                 1.0
shops                     1.0
aid_centers               1.0
other_infrastructure      1.0
weather_related           1.0
floods                    1.0
storm                     1.0
fire                      1.0
earthquake                1.0
cold      

In [47]:
for i in range(len(y_test.columns)):
    print(y_test.iloc[:, i], y_pred[:, i])
    # classification_report(y_test[col], y_pred[col])
    # print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

19464    1
373      1
7204     1
15823    1
24591    1
        ..
21308    1
1337     1
24169    1
25356    1
6643     1
Name: related, Length: 6545, dtype: int64 [1 1 1 ... 1 1 1]
19464    0
373      0
7204     0
15823    0
24591    0
        ..
21308    0
1337     0
24169    0
25356    0
6643     0
Name: request, Length: 6545, dtype: int64 [0 0 0 ... 0 0 0]
19464    0
373      0
7204     0
15823    0
24591    0
        ..
21308    0
1337     0
24169    0
25356    0
6643     0
Name: offer, Length: 6545, dtype: int64 [0 0 0 ... 0 0 0]
19464    0
373      0
7204     0
15823    0
24591    0
        ..
21308    0
1337     0
24169    0
25356    0
6643     0
Name: aid_related, Length: 6545, dtype: int64 [0 0 0 ... 0 0 0]
19464    0
373      0
7204     0
15823    0
24591    0
        ..
21308    0
1337     0
24169    0
25356    0
6643     0
Name: medical_help, Length: 6545, dtype: int64 [0 0 0 ... 0 0 0]
19464    0
373      0
7204     0
15823    0
24591    0
        ..
21308    0
1337     0


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.